In [ ]:
import netCDF4
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
from tqdm import tqdm
from shapely.geometry import Point
from scipy.stats import skew, kurtosis, entropy
from scipy.fft import fft
from sklearn.preprocessing import MinMaxScaler
import os
from pycaret.classification import setup, compare_models, tune_model, finalize_model, save_model, plot_model, evaluate_model, dashboard, save_experiment, blend_models, get_config
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split

In [ ]:
ROOT_DIR = 'E:/data/RONGOWAI_L1_SDR_V1.0/' # Change this to your root directory

In [ ]:
class SurfaceTypeUtils:
    surface_type_dict = {
        -1: "Ocean",
        0: "NaN",
        1: "Artifical",
        2: "Barely vegetated",
        3: "Inland water",
        4: "Crop",
        5: "Grass",
        6: "Shrub",
        7: "Forest"
    }
    ddm_antennas = {
        0: 'None',
        1: 'Zenith',
        2: 'LHCP',
        3: 'RHCP',
    }


class GeoUtils:
    def __init__(self, world_shapefile_path):
        self.world = gpd.read_file(world_shapefile_path)

    @staticmethod
    def add_seconds(time, seconds):
        timestamp = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
        new_timestamp = timestamp + timedelta(seconds=seconds)
        return new_timestamp.strftime("%Y-%m-%d %H:%M:%S")

    def is_land(self, lat, lon):
        point = Point(lon, lat)
        return any(self.world.contains(point))

    @staticmethod
    def check_ocean_and_land(lst):
        has_ocean = -1 in lst
        has_land = any(1 <= num <= 7 for num in lst)
        return has_ocean and has_land

    @staticmethod
    def fill_and_filter(arr):
        mask_all_nan = np.all(np.isnan(arr), axis=(2, 3))
        arr_filled = arr.copy()
        for i in range(arr.shape[0]):
            nan_indices = np.where(mask_all_nan[i])[0]
            if len(nan_indices) > 0:
                valid_indices = np.where(~mask_all_nan[i])[0]
                if len(valid_indices) > 0:
                    mean_matrix = np.nanmean(arr[i, valid_indices, :, :], axis=0)
                    arr_filled[i, nan_indices, :, :] = mean_matrix
        mask_discard = np.all(mask_all_nan, axis=1)
        arr_filtered = arr_filled[~mask_discard]
        return arr_filtered, list(np.where(mask_discard.astype(int) == 1)[0])

In [ ]:
import os
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.gridspec import GridSpec
from collections import defaultdict
import seaborn as sns

class NetCDFPreprocessor:
    def __init__(self, root_dir, preprocessing_method=str):
        self.root_dir = root_dir
        self.netcdf_file_list = os.listdir(root_dir)
        self.preprocessing_method = preprocessing_method
        if self.preprocessing_method not in ['filtered', 'with_lat_lons', 'unfiltered']:
            raise ValueError("Invalid preprocessing method. Choose from 'filtered', 'with_lat_lons', or 'unfiltered'.")
        
        # Statistiche sui picchi
        self.peak_statistics = defaultdict(list)
        self.quadrant_stats = defaultdict(int)
        
    @staticmethod
    def check_integrity(f):
        """Check integrity of the netCDF file"""
        if not isinstance(f, netCDF4.Dataset):
            raise ValueError("Input must be a netCDF4.Dataset object")
        if 'raw_counts' not in f.variables:
            raise KeyError("The netCDF file does not contain 'raw_counts' variable")
        if 'sp_alt' not in f.variables or 'sp_inc_angle' not in f.variables:
            raise KeyError("The netCDF file does not contain 'sp_alt' or 'sp_inc_angle' variables")
        if 'sp_rx_gain_copol' not in f.variables or 'sp_rx_gain_xpol' not in f.variables or 'ddm_snr' not in f.variables:
            raise KeyError("The netCDF file does not contain 'sp_rx_gain_copol', 'sp_rx_gain_xpol' or 'ddm_snr' variables")
        if 'sp_lat' not in f.variables or 'sp_lon' not in f.variables:
            raise KeyError("The netCDF file does not contain 'sp_lat' or 'sp_lon' variables")
        if 'sp_surface_type' not in f.variables:
            raise KeyError("The netCDF file does not contain 'sp_surface_type' variable")
        if 'ac_alt' not in f.variables:
            raise KeyError("The netCDF file does not contain 'ac_alt' variable")
        if f.variables['raw_counts'].ndim != 4:
            raise ValueError("The 'raw_counts' variable must have 4 dimensions")
    
    def preprocess(self, f):
        """ Preprocess the netCDF file and return fit data and labels """
        # Check integrity of the netCDF file
        self.check_integrity(f)
        
        raw_counts = f.variables['raw_counts'][:]
        ac_alt = f.variables['ac_alt'][:]
        sp_alt = f.variables['sp_alt'][:]
        copol = f.variables['sp_rx_gain_copol'][:]
        xpol = f.variables['sp_rx_gain_xpol'][:]
        snr = f.variables['ddm_snr'][:]
        sp_inc_angle = f.variables['sp_inc_angle'][:]
        
        distance_2d = (ac_alt[:, np.newaxis] - sp_alt) / np.cos(np.deg2rad(sp_inc_angle))
        
        # Filtering mask
        keep_mask = (
            (copol >= 5) & # SP copolarized gain
            (xpol >= 5) & # SP cross-polarized gain
            (snr > 0) & # Positive signal-to-Noise Ratio
            (distance_2d >= 2000) & # SP distance min
            (distance_2d <= 10000) & # SP distance max
            ~np.isnan(copol) &
            ~np.isnan(xpol) &
            ~np.isnan(snr) &
            ~np.isnan(distance_2d)
        )
        
        output_array = np.full(raw_counts.shape, np.nan, dtype=np.float32)
        i_indices, j_indices = np.where(keep_mask)
        output_array[i_indices, j_indices] = raw_counts[i_indices, j_indices]
        
        return output_array, keep_mask
    
    def find_peak_in_ddm(self, ddm):
        """
        Trova il picco (valore massimo) nella DDM
        
        Args:
            ddm: delay-doppler map (2D array)
        
        Returns:
            peak_coords: tuple (row, col) delle coordinate del picco
            peak_value: valore del picco
        """
        # Gestisce i NaN
        valid_ddm = np.nan_to_num(ddm, nan=0)
        
        # Trova il picco
        peak_value = np.nanmax(valid_ddm)
        peak_coords = np.unravel_index(np.argmax(valid_ddm), valid_ddm.shape)
        
        return peak_coords, peak_value
    
    def get_quadrant_containing_peak(self, peak_coords, ddm_shape, quadrant_size=(4, 8)):
        """
        Identifica il quadrante 4x8 che contiene il picco
        
        Args:
            peak_coords: tuple (row, col) delle coordinate del picco
            ddm_shape: tuple con le dimensioni della DDM
            quadrant_size: tuple (height, width) del quadrante
        
        Returns:
            quadrant_coords: dizionario con le coordinate del quadrante
        """
        row, col = peak_coords
        q_height, q_width = quadrant_size
        
        # Calcola l'indice del quadrante
        quadrant_row = row // q_height
        quadrant_col = col // q_width
        
        # Calcola i bounds del quadrante
        start_row = quadrant_row * q_height
        end_row = min(start_row + q_height, ddm_shape[0])
        start_col = quadrant_col * q_width
        end_col = min(start_col + q_width, ddm_shape[1])
        
        return {
            'quadrant_index': (quadrant_row, quadrant_col),
            'bounds': {
                'row_start': start_row,
                'row_end': end_row,
                'col_start': start_col,
                'col_end': end_col
            },
            'peak_local_coords': (row - start_row, col - start_col)
        }
    
    def divide_ddm_into_quadrants(self, ddm_shape, quadrant_size=(4, 8)):
        """
        Divide l'intera DDM in quadranti 4x8
        
        Args:
            ddm_shape: tuple con le dimensioni della DDM
            quadrant_size: tuple (height, width) del quadrante
        
        Returns:
            quadrants: lista di dizionari con le informazioni sui quadranti
        """
        height, width = ddm_shape
        q_height, q_width = quadrant_size
        
        quadrants = []
        
        # Calcola il numero di quadranti
        n_rows = (height + q_height - 1) // q_height
        n_cols = (width + q_width - 1) // q_width
        
        for i in range(n_rows):
            for j in range(n_cols):
                start_row = i * q_height
                end_row = min(start_row + q_height, height)
                start_col = j * q_width
                end_col = min(start_col + q_width, width)
                
                quadrants.append({
                    'index': (i, j),
                    'bounds': {
                        'row_start': start_row,
                        'row_end': end_row,
                        'col_start': start_col,
                        'col_end': end_col
                    }
                })
        
        return quadrants
    
    def visualize_ddm_with_peak(self, ddm, peak_coords, quadrant_info, title="DDM with Peak Location"):
        """
        Visualizza la DDM con il picco evidenziato e i quadranti
        
        Args:
            ddm: delay-doppler map
            peak_coords: coordinate del picco
            quadrant_info: informazioni sul quadrante contenente il picco
            title: titolo del plot
        """
        fig = plt.figure(figsize=(15, 10))
        gs = GridSpec(2, 3, figure=fig)
        
        # 1. DDM originale con picco evidenziato
        ax1 = fig.add_subplot(gs[0, 0])
        im1 = ax1.imshow(ddm, cmap='hot', aspect='auto')
        ax1.scatter(peak_coords[1], peak_coords[0], c='blue', s=100, marker='x', linewidths=3)
        ax1.set_title('DDM con Picco')
        ax1.set_xlabel('Doppler bins')
        ax1.set_ylabel('Delay bins')
        plt.colorbar(im1, ax=ax1)
        
        # 2. DDM con quadranti
        ax2 = fig.add_subplot(gs[0, 1])
        im2 = ax2.imshow(ddm, cmap='hot', aspect='auto', alpha=0.7)
        
        # Disegna tutti i quadranti
        quadrants = self.divide_ddm_into_quadrants(ddm.shape)
        for quad in quadrants:
            bounds = quad['bounds']
            rect = patches.Rectangle(
                (bounds['col_start'], bounds['row_start']),
                bounds['col_end'] - bounds['col_start'],
                bounds['row_end'] - bounds['row_start'],
                linewidth=1, edgecolor='white', facecolor='none', alpha=0.5
            )
            ax2.add_patch(rect)
        
        # Evidenzia il quadrante con il picco
        peak_bounds = quadrant_info['bounds']
        rect_peak = patches.Rectangle(
            (peak_bounds['col_start'], peak_bounds['row_start']),
            peak_bounds['col_end'] - peak_bounds['col_start'],
            peak_bounds['row_end'] - peak_bounds['row_start'],
            linewidth=3, edgecolor='lime', facecolor='none'
        )
        ax2.add_patch(rect_peak)
        ax2.scatter(peak_coords[1], peak_coords[0], c='blue', s=100, marker='x', linewidths=3)
        ax2.set_title('DDM con Quadranti (4x8)')
        ax2.set_xlabel('Doppler bins')
        ax2.set_ylabel('Delay bins')
        plt.colorbar(im2, ax=ax2)
        
        # 3. Zoom sul quadrante del picco
        ax3 = fig.add_subplot(gs[0, 2])
        peak_quadrant = ddm[
            peak_bounds['row_start']:peak_bounds['row_end'],
            peak_bounds['col_start']:peak_bounds['col_end']
        ]
        im3 = ax3.imshow(peak_quadrant, cmap='hot', aspect='auto')
        local_coords = quadrant_info['peak_local_coords']
        ax3.scatter(local_coords[1], local_coords[0], c='blue', s=100, marker='x', linewidths=3)
        ax3.set_title(f'Quadrante Picco {quadrant_info["quadrant_index"]}')
        ax3.set_xlabel('Doppler bins (locale)')
        ax3.set_ylabel('Delay bins (locale)')
        plt.colorbar(im3, ax=ax3)
        
        # 4. Profilo delay (attraverso il picco)
        ax4 = fig.add_subplot(gs[1, 0])
        delay_profile = ddm[:, peak_coords[1]]
        ax4.plot(delay_profile, 'b-', linewidth=2)
        ax4.axvline(x=peak_coords[0], color='r', linestyle='--', label=f'Picco a delay={peak_coords[0]}')
        ax4.set_xlabel('Delay bins')
        ax4.set_ylabel('Potenza')
        ax4.set_title('Profilo Delay (attraverso il picco)')
        ax4.grid(True, alpha=0.3)
        ax4.legend()
        
        # 5. Profilo Doppler (attraverso il picco)
        ax5 = fig.add_subplot(gs[1, 1])
        doppler_profile = ddm[peak_coords[0], :]
        ax5.plot(doppler_profile, 'b-', linewidth=2)
        ax5.axvline(x=peak_coords[1], color='r', linestyle='--', label=f'Picco a doppler={peak_coords[1]}')
        ax5.set_xlabel('Doppler bins')
        ax5.set_ylabel('Potenza')
        ax5.set_title('Profilo Doppler (attraverso il picco)')
        ax5.grid(True, alpha=0.3)
        ax5.legend()
        
        # 6. Info testuale
        ax6 = fig.add_subplot(gs[1, 2])
        ax6.axis('off')
        info_text = f"""
        Informazioni Picco:
        - Posizione: ({peak_coords[0]}, {peak_coords[1]})
        - Valore: {ddm[peak_coords]:.2f}
        - Quadrante: {quadrant_info['quadrant_index']}
        - Bounds quadrante:
          Rows: [{peak_bounds['row_start']}, {peak_bounds['row_end']}]
          Cols: [{peak_bounds['col_start']}, {peak_bounds['col_end']}]
        - Posizione locale nel quadrante: {local_coords}
        """
        ax6.text(0.1, 0.5, info_text, transform=ax6.transAxes, 
                fontsize=10, verticalalignment='center', family='monospace')
        
        plt.suptitle(title, fontsize=14, fontweight='bold')
        plt.tight_layout()
        return fig
    
    def update_peak_statistics(self, peak_coords, quadrant_info, ddm_shape):
        """
        Aggiorna le statistiche sui picchi per analisi successive
        
        Args:
            peak_coords: coordinate del picco
            quadrant_info: informazioni sul quadrante
            ddm_shape: dimensioni della DDM
        """
        # Salva posizione assoluta del picco
        self.peak_statistics['peak_row'].append(peak_coords[0])
        self.peak_statistics['peak_col'].append(peak_coords[1])
        
        # Salva posizione normalizzata (0-1)
        self.peak_statistics['peak_row_norm'].append(peak_coords[0] / ddm_shape[0])
        self.peak_statistics['peak_col_norm'].append(peak_coords[1] / ddm_shape[1])
        
        # Conta occorrenze per quadrante
        quad_idx = quadrant_info['quadrant_index']
        self.quadrant_stats[quad_idx] += 1
    
    def visualize_ddm_quadrant_grid(self, ddm, peak_coords=None, quadrant_size=(4, 8), 
                                     show_values=True, colormap='hot'):
        """
        Visualizza DDM con griglia dettagliata dei quadranti
        
        Args:
            ddm: delay-doppler map
            peak_coords: coordinate del picco (opzionale)
            quadrant_size: dimensioni del quadrante (height, width)
            show_values: se True, mostra i valori medi per quadrante
            colormap: colormap da utilizzare
        """
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        
        # Ottieni tutti i quadranti
        quadrants = self.divide_ddm_into_quadrants(ddm.shape, quadrant_size)
        n_rows = max(q['index'][0] for q in quadrants) + 1
        n_cols = max(q['index'][1] for q in quadrants) + 1
        
        # 1. DDM con griglia numerata
        ax1 = axes[0]
        im1 = ax1.imshow(ddm, cmap=colormap, aspect='auto', alpha=0.8)
        
        # Disegna griglia e numera i quadranti
        for quad in quadrants:
            bounds = quad['bounds']
            idx = quad['index']
            
            # Rettangolo del quadrante
            rect = patches.Rectangle(
                (bounds['col_start'], bounds['row_start']),
                bounds['col_end'] - bounds['col_start'],
                bounds['row_end'] - bounds['row_start'],
                linewidth=2, edgecolor='white', facecolor='none'
            )
            ax1.add_patch(rect)
            
            # Numero del quadrante
            center_x = (bounds['col_start'] + bounds['col_end']) / 2
            center_y = (bounds['row_start'] + bounds['row_end']) / 2
            ax1.text(center_x, center_y, f'{idx[0]},{idx[1]}', 
                    ha='center', va='center', color='yellow', 
                    fontsize=8, weight='bold',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='black', alpha=0.5))
        
        # Evidenzia picco se fornito
        if peak_coords is not None:
            ax1.scatter(peak_coords[1], peak_coords[0], c='lime', s=150, 
                       marker='*', edgecolors='white', linewidths=2)
            # Evidenzia quadrante del picco
            peak_quad = self.get_quadrant_containing_peak(peak_coords, ddm.shape, quadrant_size)
            peak_bounds = peak_quad['bounds']
            rect_peak = patches.Rectangle(
                (peak_bounds['col_start'], peak_bounds['row_start']),
                peak_bounds['col_end'] - peak_bounds['col_start'],
                peak_bounds['row_end'] - peak_bounds['row_start'],
                linewidth=3, edgecolor='lime', facecolor='lime', alpha=0.2
            )
            ax1.add_patch(rect_peak)
        
        ax1.set_title(f'DDM con Griglia Quadranti ({quadrant_size[0]}×{quadrant_size[1]})')
        ax1.set_xlabel('Doppler bins')
        ax1.set_ylabel('Delay bins')
        plt.colorbar(im1, ax=ax1, label='Potenza')
        
        # 2. Mappa dei valori medi per quadrante
        ax2 = axes[1]
        quadrant_means = np.zeros((n_rows, n_cols))
        quadrant_maxs = np.zeros((n_rows, n_cols))
        
        for quad in quadrants:
            bounds = quad['bounds']
            idx = quad['index']
            
            # Estrai il quadrante
            quad_data = ddm[bounds['row_start']:bounds['row_end'],
                           bounds['col_start']:bounds['col_end']]
            
            # Calcola statistiche
            quadrant_means[idx[0], idx[1]] = np.nanmean(quad_data)
            quadrant_maxs[idx[0], idx[1]] = np.nanmax(quad_data)
        
        im2 = ax2.imshow(quadrant_means, cmap='viridis', aspect='auto')
        
        # Aggiungi valori testuali
        if show_values:
            for i in range(n_rows):
                for j in range(n_cols):
                    text_color = 'white' if quadrant_means[i, j] < np.nanmean(quadrant_means) else 'black'
                    ax2.text(j, i, f'{quadrant_means[i, j]:.1f}',
                           ha='center', va='center', color=text_color, fontsize=9)
        
        ax2.set_title('Potenza Media per Quadrante')
        ax2.set_xlabel('Indice Colonna Quadrante')
        ax2.set_ylabel('Indice Riga Quadrante')
        ax2.set_xticks(range(n_cols))
        ax2.set_yticks(range(n_rows))
        plt.colorbar(im2, ax=ax2, label='Potenza Media')
        
        # 3. Mappa dei valori massimi per quadrante
        ax3 = axes[2]
        im3 = ax3.imshow(quadrant_maxs, cmap='plasma', aspect='auto')
        
        # Aggiungi valori testuali
        if show_values:
            for i in range(n_rows):
                for j in range(n_cols):
                    text_color = 'white' if quadrant_maxs[i, j] < np.nanmean(quadrant_maxs) else 'black'
                    ax3.text(j, i, f'{quadrant_maxs[i, j]:.1f}',
                           ha='center', va='center', color=text_color, fontsize=9)
        
        # Evidenzia il quadrante con il valore massimo globale
        max_idx = np.unravel_index(np.argmax(quadrant_maxs), quadrant_maxs.shape)
        rect_max = patches.Rectangle(
            (max_idx[1]-0.45, max_idx[0]-0.45), 0.9, 0.9,
            linewidth=3, edgecolor='red', facecolor='none'
        )
        ax3.add_patch(rect_max)
        
        ax3.set_title('Potenza Massima per Quadrante')
        ax3.set_xlabel('Indice Colonna Quadrante')
        ax3.set_ylabel('Indice Riga Quadrante')
        ax3.set_xticks(range(n_cols))
        ax3.set_yticks(range(n_rows))
        plt.colorbar(im3, ax=ax3, label='Potenza Massima')
        
        plt.suptitle('Analisi Dettagliata Quadranti DDM', fontsize=14, fontweight='bold')
        plt.tight_layout()
        return fig
    
    def visualize_multiple_ddm_quadrants(self, ddm_list, peak_coords_list=None, 
                                        quadrant_size=(4, 8), max_display=6):
        """
        Visualizza multiple DDM con i loro quadranti in una griglia
        
        Args:
            ddm_list: lista di DDM da visualizzare
            peak_coords_list: lista delle coordinate dei picchi (opzionale)
            quadrant_size: dimensioni del quadrante
            max_display: numero massimo di DDM da visualizzare
        """
        n_ddms = min(len(ddm_list), max_display)
        n_cols = 3
        n_rows = (n_ddms + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        
        for idx in range(n_ddms):
            row = idx // n_cols
            col = idx % n_cols
            ax = axes[row, col]
            
            ddm = ddm_list[idx]
            im = ax.imshow(ddm, cmap='hot', aspect='auto', alpha=0.9)
            
            # Disegna quadranti
            quadrants = self.divide_ddm_into_quadrants(ddm.shape, quadrant_size)
            for quad in quadrants:
                bounds = quad['bounds']
                rect = patches.Rectangle(
                    (bounds['col_start'], bounds['row_start']),
                    bounds['col_end'] - bounds['col_start'],
                    bounds['row_end'] - bounds['row_start'],
                    linewidth=1, edgecolor='cyan', facecolor='none', alpha=0.7
                )
                ax.add_patch(rect)
            
            # Aggiungi picco se disponibile
            if peak_coords_list and idx < len(peak_coords_list):
                peak = peak_coords_list[idx]
                if peak is not None:
                    ax.scatter(peak[1], peak[0], c='lime', s=100, 
                             marker='*', edgecolors='white', linewidths=1.5)
                    
                    # Evidenzia quadrante del picco
                    peak_quad = self.get_quadrant_containing_peak(peak, ddm.shape, quadrant_size)
                    peak_bounds = peak_quad['bounds']
                    rect_peak = patches.Rectangle(
                        (peak_bounds['col_start'], peak_bounds['row_start']),
                        peak_bounds['col_end'] - peak_bounds['col_start'],
                        peak_bounds['row_end'] - peak_bounds['row_start'],
                        linewidth=2, edgecolor='lime', facecolor='none'
                    )
                    ax.add_patch(rect_peak)
                    
                    ax.set_title(f'DDM {idx+1} - Picco in Q{peak_quad["quadrant_index"]}', 
                               fontsize=10)
            else:
                ax.set_title(f'DDM {idx+1}', fontsize=10)
            
            ax.set_xlabel('Doppler', fontsize=8)
            ax.set_ylabel('Delay', fontsize=8)
            ax.tick_params(labelsize=7)
        
        # Nascondi assi vuoti
        for idx in range(n_ddms, n_rows * n_cols):
            row = idx // n_cols
            col = idx % n_cols
            axes[row, col].axis('off')
        
        plt.suptitle(f'Visualizzazione Multiple DDM con Quadranti {quadrant_size[0]}×{quadrant_size[1]}', 
                    fontsize=14, fontweight='bold')
        plt.tight_layout()
        return fig
    
    def create_quadrant_analysis_report(self, ddm, peak_coords, save_path=None):
        """
        Crea un report completo dell'analisi dei quadranti per una DDM
        
        Args:
            ddm: delay-doppler map
            peak_coords: coordinate del picco
            save_path: percorso dove salvare il report (opzionale)
        """
        fig = plt.figure(figsize=(20, 12))
        gs = GridSpec(3, 4, figure=fig, hspace=0.3, wspace=0.3)
        
        # 1. DDM originale
        ax1 = fig.add_subplot(gs[0, :2])
        im1 = ax1.imshow(ddm, cmap='hot', aspect='auto')
        ax1.set_title('DDM Originale', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Doppler bins')
        ax1.set_ylabel('Delay bins')
        plt.colorbar(im1, ax=ax1)
        
        # 2. DDM con quadranti e picco
        ax2 = fig.add_subplot(gs[0, 2:])
        im2 = ax2.imshow(ddm, cmap='hot', aspect='auto', alpha=0.7)
        
        quadrants = self.divide_ddm_into_quadrants(ddm.shape, quadrant_size=(4, 8))
        colors = plt.cm.Set3(np.linspace(0, 1, len(quadrants)))
        
        for i, quad in enumerate(quadrants):
            bounds = quad['bounds']
            rect = patches.Rectangle(
                (bounds['col_start'], bounds['row_start']),
                bounds['col_end'] - bounds['col_start'],
                bounds['row_end'] - bounds['row_start'],
                linewidth=2, edgecolor=colors[i % len(colors)], 
                facecolor='none', alpha=0.8
            )
            ax2.add_patch(rect)
        
        # Evidenzia picco e suo quadrante
        ax2.scatter(peak_coords[1], peak_coords[0], c='red', s=200, 
                   marker='*', edgecolors='yellow', linewidths=2)
        peak_quad = self.get_quadrant_containing_peak(peak_coords, ddm.shape, (4, 8))
        peak_bounds = peak_quad['bounds']
        rect_peak = patches.Rectangle(
            (peak_bounds['col_start'], peak_bounds['row_start']),
            peak_bounds['col_end'] - peak_bounds['col_start'],
            peak_bounds['row_end'] - peak_bounds['row_start'],
            linewidth=3, edgecolor='yellow', facecolor='yellow', alpha=0.2
        )
        ax2.add_patch(rect_peak)
        
        ax2.set_title('DDM con Quadranti Colorati e Picco', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Doppler bins')
        ax2.set_ylabel('Delay bins')
        plt.colorbar(im2, ax=ax2)
        
        # 3-6. Analisi per quadrante (4 quadranti esempio)
        for i in range(4):
            ax = fig.add_subplot(gs[1, i])
            if i < len(quadrants):
                bounds = quadrants[i]['bounds']
                quad_data = ddm[bounds['row_start']:bounds['row_end'],
                              bounds['col_start']:bounds['col_end']]
                im = ax.imshow(quad_data, cmap='hot', aspect='auto')
                ax.set_title(f'Q{quadrants[i]["index"]}', fontsize=10)
                ax.set_xlabel('Doppler', fontsize=8)
                ax.set_ylabel('Delay', fontsize=8)
                plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            else:
                ax.axis('off')
        
        # 7. Statistiche per quadrante
        ax7 = fig.add_subplot(gs[2, :2])
        quad_stats = []
        for quad in quadrants:
            bounds = quad['bounds']
            quad_data = ddm[bounds['row_start']:bounds['row_end'],
                          bounds['col_start']:bounds['col_end']]
            quad_stats.append({
                'index': f"{quad['index'][0]},{quad['index'][1]}",
                'mean': np.nanmean(quad_data),
                'max': np.nanmax(quad_data),
                'std': np.nanstd(quad_data)
            })
        
        # Crea bar plot
        indices = [q['index'] for q in quad_stats]
        means = [q['mean'] for q in quad_stats]
        maxs = [q['max'] for q in quad_stats]
        
        x = np.arange(len(indices))
        width = 0.35
        
        bars1 = ax7.bar(x - width/2, means, width, label='Media', alpha=0.8)
        bars2 = ax7.bar(x + width/2, maxs, width, label='Massimo', alpha=0.8)
        
        ax7.set_xlabel('Quadrante', fontsize=10)
        ax7.set_ylabel('Potenza', fontsize=10)
        ax7.set_title('Statistiche per Quadrante', fontsize=12, fontweight='bold')
        ax7.set_xticks(x)
        ax7.set_xticklabels(indices, rotation=45, ha='right', fontsize=7)
        ax7.legend()
        ax7.grid(True, alpha=0.3)
        
        # 8. Info testuale dettagliata
        ax8 = fig.add_subplot(gs[2, 2:])
        ax8.axis('off')
        
        info_text = f"""
        REPORT ANALISI QUADRANTI
        ========================
        
        Dimensioni DDM: {ddm.shape}
        Dimensioni Quadrante: 4×8
        Numero Totale Quadranti: {len(quadrants)}
        
        PICCO RILEVATO:
        - Posizione: ({peak_coords[0]}, {peak_coords[1]})
        - Valore: {ddm[peak_coords]:.2f}
        - Quadrante: {peak_quad['quadrant_index']}
        - Posizione locale: {peak_quad['peak_local_coords']}
        
        STATISTICHE GLOBALI:
        - Media DDM: {np.nanmean(ddm):.2f}
        - Max DDM: {np.nanmax(ddm):.2f}
        - Std DDM: {np.nanstd(ddm):.2f}
        - Quadranti non vuoti: {sum(1 for q in quad_stats if q['mean'] > 0)}
        """
        
        ax8.text(0.05, 0.95, info_text, transform=ax8.transAxes,
                fontsize=9, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round,pad=1', facecolor='lightgray', alpha=0.8))
        
        plt.suptitle('Report Completo Analisi Quadranti DDM', fontsize=16, fontweight='bold')
        
        if save_path:
            plt.savefig(save_path, dpi=150, bbox_inches='tight')
            print(f"Report salvato in: {save_path}")
        
        return fig
    
    def analyze_quadrant_energy_distribution(self, ddm, quadrant_size=(4, 8)):
        """
        Analizza la distribuzione dell'energia tra i quadranti
        
        Args:
            ddm: delay-doppler map
            quadrant_size: dimensioni del quadrante
        
        Returns:
            analysis: dizionario con analisi dettagliata
        """
        quadrants = self.divide_ddm_into_quadrants(ddm.shape, quadrant_size)
        total_energy = np.nansum(ddm)
        
        analysis = {
            'quadrants': [],
            'total_energy': total_energy,
            'energy_distribution': [],
            'concentration_index': 0  # Indice di concentrazione dell'energia (0-1)
        }
        
        energies = []
        
        for quad in quadrants:
            bounds = quad['bounds']
            quad_data = ddm[bounds['row_start']:bounds['row_end'],
                          bounds['col_start']:bounds['col_end']]
            
            quad_energy = np.nansum(quad_data)
            quad_mean = np.nanmean(quad_data)
            quad_max = np.nanmax(quad_data)
            quad_std = np.nanstd(quad_data)
            
            energy_percentage = (quad_energy / total_energy * 100) if total_energy > 0 else 0
            
            quad_analysis = {
                'index': quad['index'],
                'energy': quad_energy,
                'energy_percentage': energy_percentage,
                'mean': quad_mean,
                'max': quad_max,
                'std': quad_std,
                'bounds': bounds
            }
            
            analysis['quadrants'].append(quad_analysis)
            energies.append(quad_energy)
        
        # Calcola indice di concentrazione (Gini-like coefficient)
        energies = np.array(energies)
        energies_sorted = np.sort(energies)
        n = len(energies)
        cumsum = np.cumsum(energies_sorted)
        analysis['concentration_index'] = (n + 1 - 2 * np.sum(cumsum) / cumsum[-1]) / n if cumsum[-1] > 0 else 0
        
        # Identifica quadranti dominanti (top 20% dell'energia)
        threshold = np.percentile(energies, 80)
        analysis['dominant_quadrants'] = [
            q for q in analysis['quadrants'] if q['energy'] >= threshold
        ]
        
        return analysis
    
    def visualize_energy_distribution(self, ddm, quadrant_size=(4, 8)):
        """
        Visualizza la distribuzione dell'energia tra i quadranti
        
        Args:
            ddm: delay-doppler map
            quadrant_size: dimensioni del quadrante
        """
        analysis = self.analyze_quadrant_energy_distribution(ddm, quadrant_size)
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        
        # 1. DDM con energia percentuale per quadrante
        ax1 = axes[0, 0]
        im1 = ax1.imshow(ddm, cmap='hot', aspect='auto', alpha=0.6)
        
        for quad_info in analysis['quadrants']:
            bounds = quad_info['bounds']
            center_x = (bounds['col_start'] + bounds['col_end']) / 2
            center_y = (bounds['row_start'] + bounds['row_end']) / 2
            
            # Colore in base all'energia
            if quad_info['energy_percentage'] > 10:
                color = 'red'
                fontweight = 'bold'
            elif quad_info['energy_percentage'] > 5:
                color = 'orange'
                fontweight = 'normal'
            else:
                color = 'white'
                fontweight = 'normal'
            
            # Disegna bordo quadrante
            rect = patches.Rectangle(
                (bounds['col_start'], bounds['row_start']),
                bounds['col_end'] - bounds['col_start'],
                bounds['row_end'] - bounds['row_start'],
                linewidth=1, edgecolor='white', facecolor='none', alpha=0.5
            )
            ax1.add_patch(rect)
            
            # Mostra percentuale energia
            ax1.text(center_x, center_y, f'{quad_info["energy_percentage"]:.1f}%',
                    ha='center', va='center', color=color, fontsize=7,
                    fontweight=fontweight,
                    bbox=dict(boxstyle='round,pad=0.2', facecolor='black', alpha=0.6))
        
        ax1.set_title('Distribuzione Energia per Quadrante (%)', fontsize=11)
        ax1.set_xlabel('Doppler bins')
        ax1.set_ylabel('Delay bins')
        plt.colorbar(im1, ax=ax1)
        
        # 2. Heatmap energia per quadrante
        ax2 = axes[0, 1]
        n_rows = max(q['index'][0] for q in analysis['quadrants']) + 1
        n_cols = max(q['index'][1] for q in analysis['quadrants']) + 1
        energy_matrix = np.zeros((n_rows, n_cols))
        
        for quad_info in analysis['quadrants']:
            idx = quad_info['index']
            energy_matrix[idx[0], idx[1]] = quad_info['energy']
        
        im2 = ax2.imshow(energy_matrix, cmap='YlOrRd', aspect='auto')
        ax2.set_title('Heatmap Energia Totale', fontsize=11)
        ax2.set_xlabel('Indice Colonna')
        ax2.set_ylabel('Indice Riga')
        
        # Aggiungi valori
        for i in range(n_rows):
            for j in range(n_cols):
                text_color = 'white' if energy_matrix[i, j] > np.mean(energy_matrix) else 'black'
                ax2.text(j, i, f'{energy_matrix[i, j]:.0f}',
                       ha='center', va='center', color=text_color, fontsize=8)
        
        plt.colorbar(im2, ax=ax2, label='Energia')
        
        # 3. Bar plot top 10 quadranti
        ax3 = axes[0, 2]
        sorted_quads = sorted(analysis['quadrants'], 
                            key=lambda x: x['energy'], reverse=True)[:10]
        
        indices = [f"{q['index'][0]},{q['index'][1]}" for q in sorted_quads]
        energies = [q['energy'] for q in sorted_quads]
        percentages = [q['energy_percentage'] for q in sorted_quads]
        
        bars = ax3.bar(range(len(indices)), energies, color='steelblue', alpha=0.8)
        
        # Colora le barre dominanti
        for i, (bar, pct) in enumerate(zip(bars, percentages)):
            if pct > 10:
                bar.set_color('red')
            elif pct > 5:
                bar.set_color('orange')
        
        ax3.set_xlabel('Quadrante', fontsize=10)
        ax3.set_ylabel('Energia', fontsize=10)
        ax3.set_title('Top 10 Quadranti per Energia', fontsize=11)
        ax3.set_xticks(range(len(indices)))
        ax3.set_xticklabels(indices, rotation=45, ha='right', fontsize=8)
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Aggiungi percentuali sopra le barre
        for i, (bar, pct) in enumerate(zip(bars, percentages)):
            ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height(),
                    f'{pct:.1f}%', ha='center', va='bottom', fontsize=7)
        
        # 4. Distribuzione cumulativa energia
        ax4 = axes[1, 0]
        sorted_energies = sorted([q['energy'] for q in analysis['quadrants']], reverse=True)
        cumulative_pct = np.cumsum(sorted_energies) / analysis['total_energy'] * 100
        
        ax4.plot(range(1, len(cumulative_pct) + 1), cumulative_pct, 
                'b-', linewidth=2, marker='o', markersize=4)
        ax4.axhline(y=50, color='r', linestyle='--', alpha=0.5, label='50% energia')
        ax4.axhline(y=80, color='orange', linestyle='--', alpha=0.5, label='80% energia')
        ax4.set_xlabel('Numero di Quadranti', fontsize=10)
        ax4.set_ylabel('Energia Cumulativa (%)', fontsize=10)
        ax4.set_title('Distribuzione Cumulativa Energia', fontsize=11)
        ax4.grid(True, alpha=0.3)
        ax4.legend()
        
        # 5. Scatter plot energia vs posizione
        ax5 = axes[1, 1]
        x_positions = [(q['bounds']['col_start'] + q['bounds']['col_end']) / 2 
                      for q in analysis['quadrants']]
        y_positions = [(q['bounds']['row_start'] + q['bounds']['row_end']) / 2 
                      for q in analysis['quadrants']]
        energies = [q['energy'] for q in analysis['quadrants']]
        
        scatter = ax5.scatter(x_positions, y_positions, s=np.array(energies)/max(energies)*500,
                            c=energies, cmap='plasma', alpha=0.6, edgecolors='black')
        ax5.set_xlabel('Posizione Doppler (centro quadrante)', fontsize=10)
        ax5.set_ylabel('Posizione Delay (centro quadrante)', fontsize=10)
        ax5.set_title('Energia vs Posizione Spaziale', fontsize=11)
        plt.colorbar(scatter, ax=ax5, label='Energia')
        ax5.grid(True, alpha=0.3)
        
        # 6. Statistiche testuali
        ax6 = axes[1, 2]
        ax6.axis('off')
        
        n_dominant = len(analysis['dominant_quadrants'])
        dominant_energy_pct = sum(q['energy_percentage'] for q in analysis['dominant_quadrants'])
        
        stats_text = f"""
        ANALISI DISTRIBUZIONE ENERGIA
        ==============================
        
        Energia Totale: {analysis['total_energy']:.2f}
        Numero Quadranti: {len(analysis['quadrants'])}
        Indice Concentrazione: {analysis['concentration_index']:.3f}
        
        QUADRANTI DOMINANTI (top 20%):
        - Numero: {n_dominant}
        - Energia totale: {dominant_energy_pct:.1f}%
        
        STATISTICHE:
        - Energia media/quadrante: {np.mean([q['energy'] for q in analysis['quadrants']]):.2f}
        - Deviazione standard: {np.std([q['energy'] for q in analysis['quadrants']]):.2f}
        - Energia max quadrante: {max(q['energy'] for q in analysis['quadrants']):.2f}
        - Energia min quadrante: {min(q['energy'] for q in analysis['quadrants']):.2f}
        
        CONCENTRAZIONE:
        - Quadranti per 50% energia: {np.searchsorted(cumulative_pct, 50) + 1}
        - Quadranti per 80% energia: {np.searchsorted(cumulative_pct, 80) + 1}
        - Quadranti per 95% energia: {np.searchsorted(cumulative_pct, 95) + 1}
        """
        
        ax6.text(0.05, 0.95, stats_text, transform=ax6.transAxes,
                fontsize=9, verticalalignment='top', family='monospace',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))
        
        plt.suptitle('Analisi Distribuzione Energia DDM', fontsize=14, fontweight='bold')
        plt.tight_layout()
        return fig
    
    def plot_peak_statistics(self):
        """
        Visualizza le statistiche cumulative sui picchi
        """
        if not self.peak_statistics['peak_row']:
            print("Nessuna statistica disponibile. Processa prima alcuni file.")
            return
        
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        
        # 1. Istogramma posizioni delay
        ax = axes[0, 0]
        ax.hist(self.peak_statistics['peak_row'], bins=20, edgecolor='black', alpha=0.7)
        ax.set_xlabel('Delay bin')
        ax.set_ylabel('Frequenza')
        ax.set_title('Distribuzione Picchi - Delay')
        ax.grid(True, alpha=0.3)
        
        # 2. Istogramma posizioni Doppler
        ax = axes[0, 1]
        ax.hist(self.peak_statistics['peak_col'], bins=20, edgecolor='black', alpha=0.7)
        ax.set_xlabel('Doppler bin')
        ax.set_ylabel('Frequenza')
        ax.set_title('Distribuzione Picchi - Doppler')
        ax.grid(True, alpha=0.3)
        
        # 3. Scatter plot 2D delle posizioni
        ax = axes[0, 2]
        ax.scatter(self.peak_statistics['peak_col'], self.peak_statistics['peak_row'], 
                  alpha=0.5, s=30)
        ax.set_xlabel('Doppler bin')
        ax.set_ylabel('Delay bin')
        ax.set_title('Posizioni 2D dei Picchi')
        ax.grid(True, alpha=0.3)
        
        # 4. Heatmap delle occorrenze per quadrante
        ax = axes[1, 0]
        if self.quadrant_stats:
            # Crea matrice per heatmap
            max_row = max(k[0] for k in self.quadrant_stats.keys())
            max_col = max(k[1] for k in self.quadrant_stats.keys())
            heatmap_data = np.zeros((max_row + 1, max_col + 1))
            
            for (r, c), count in self.quadrant_stats.items():
                heatmap_data[r, c] = count
            
            im = ax.imshow(heatmap_data, cmap='YlOrRd', aspect='auto')
            ax.set_xlabel('Indice Colonna Quadrante')
            ax.set_ylabel('Indice Riga Quadrante')
            ax.set_title('Frequenza Picchi per Quadrante')
            plt.colorbar(im, ax=ax)
            
            # Aggiungi numeri nelle celle
            for i in range(heatmap_data.shape[0]):
                for j in range(heatmap_data.shape[1]):
                    if heatmap_data[i, j] > 0:
                        ax.text(j, i, f'{int(heatmap_data[i, j])}',
                               ha='center', va='center', color='black')
        
        # 5. Box plot per delay e doppler
        ax = axes[1, 1]
        data_to_plot = [self.peak_statistics['peak_row'], self.peak_statistics['peak_col']]
        bp = ax.boxplot(data_to_plot, labels=['Delay', 'Doppler'])
        ax.set_ylabel('Bin')
        ax.set_title('Box Plot Posizioni Picchi')
        ax.grid(True, alpha=0.3)
        
        # 6. Statistiche testuali
        ax = axes[1, 2]
        ax.axis('off')
        
        stats_text = f"""
        Statistiche Riassuntive ({len(self.peak_statistics['peak_row'])} campioni):
        
        Delay:
        - Media: {np.mean(self.peak_statistics['peak_row']):.2f}
        - Std: {np.std(self.peak_statistics['peak_row']):.2f}
        - Min: {np.min(self.peak_statistics['peak_row']):.0f}
        - Max: {np.max(self.peak_statistics['peak_row']):.0f}
        
        Doppler:
        - Media: {np.mean(self.peak_statistics['peak_col']):.2f}
        - Std: {np.std(self.peak_statistics['peak_col']):.2f}
        - Min: {np.min(self.peak_statistics['peak_col']):.0f}
        - Max: {np.max(self.peak_statistics['peak_col']):.0f}
        
        Quadrante più frequente: {max(self.quadrant_stats, key=self.quadrant_stats.get) if self.quadrant_stats else 'N/A'}
        """
        
        ax.text(0.1, 0.5, stats_text, transform=ax.transAxes,
               fontsize=10, verticalalignment='center', family='monospace')
        
        plt.suptitle('Analisi Statistica Posizioni Picchi DDM', fontsize=14, fontweight='bold')
        plt.tight_layout()
        return fig
    
    def process_file_with_peak_analysis(self, filepath, visualize=True, 
                                       visualization_type='standard', save_reports=False):
        """
        Processa un singolo file NetCDF con analisi del picco
        
        Args:
            filepath: percorso del file NetCDF
            visualize: se True, visualizza i risultati
            visualization_type: tipo di visualizzazione ('standard', 'quadrant_grid', 'report', 'all')
            save_reports: se True, salva i report come immagini
        
        Returns:
            results: dizionario con i risultati dell'analisi
        """
        with netCDF4.Dataset(filepath, 'r') as f:
            # Preprocessing standard
            output_array, keep_mask = self.preprocess(f)
            
            results = []
            ddm_list = []
            peak_list = []
            
            # Processa ogni DDM nel file
            for i in range(output_array.shape[0]):
                for j in range(output_array.shape[1]):
                    ddm = output_array[i, j, :, :]
                    
                    # Salta DDM vuote o con solo NaN
                    if np.all(np.isnan(ddm)) or np.nanmax(ddm) == 0:
                        continue
                    
                    # Trova il picco
                    peak_coords, peak_value = self.find_peak_in_ddm(ddm)
                    
                    # Identifica il quadrante
                    quadrant_info = self.get_quadrant_containing_peak(
                        peak_coords, ddm.shape, quadrant_size=(4, 8)
                    )
                    
                    # Aggiorna statistiche
                    self.update_peak_statistics(peak_coords, quadrant_info, ddm.shape)
                    
                    # Salva per visualizzazione multipla
                    ddm_list.append(ddm)
                    peak_list.append(peak_coords)
                    
                    # Visualizza se richiesto
                    if visualize and visualization_type in ['standard', 'all']:
                        fig = self.visualize_ddm_with_peak(
                            ddm, peak_coords, quadrant_info,
                            title=f"DDM [{i},{j}] - File: {os.path.basename(filepath)}"
                        )
                        plt.show()
                    
                    if visualize and visualization_type in ['quadrant_grid', 'all']:
                        fig = self.visualize_ddm_quadrant_grid(
                            ddm, peak_coords, quadrant_size=(4, 8)
                        )
                        plt.show()
                    
                    if visualize and visualization_type in ['report', 'all']:
                        save_path = None
                        if save_reports:
                            save_path = f"ddm_report_{os.path.basename(filepath)}_{i}_{j}.png"
                        fig = self.create_quadrant_analysis_report(
                            ddm, peak_coords, save_path
                        )
                        plt.show()
                    
                    results.append({
                        'ddm_index': (i, j),
                        'peak_coords': peak_coords,
                        'peak_value': peak_value,
                        'quadrant_info': quadrant_info,
                        'ddm': ddm  # Aggiungi anche la DDM stessa per analisi successive
                    })
            
            # Visualizzazione multipla alla fine
            if visualize and len(ddm_list) > 0 and visualization_type in ['multiple', 'all']:
                fig = self.visualize_multiple_ddm_quadrants(
                    ddm_list, peak_list, quadrant_size=(4, 8)
                )
                plt.show()
            
            return results
    
    def process_all_files(self, max_files=None, visualize=False):
        """
        Processa tutti i file NetCDF nella directory
        
        Args:
            max_files: numero massimo di file da processare (None per tutti)
            visualize: se True, visualizza ogni DDM
        
        Returns:
            all_results: lista con tutti i risultati
        """
        all_results = []
        files_to_process = self.netcdf_file_list[:max_files] if max_files else self.netcdf_file_list
        
        for idx, filename in enumerate(files_to_process):
            if filename.endswith('.nc'):
                filepath = os.path.join(self.root_dir, filename)
                print(f"Processing file {idx+1}/{len(files_to_process)}: {filename}")
                
                try:
                    results = self.process_file_with_peak_analysis(filepath, visualize=visualize)
                    all_results.extend(results)
                except Exception as e:
                    print(f"Error processing {filename}: {str(e)}")
                    continue
        
        # Mostra statistiche finali
        if all_results:
            print(f"\nProcessati {len(all_results)} DDM totali")
            self.plot_peak_statistics()
            plt.show()
        
        return all_results




In [ ]:
# Esempio di utilizzo
if __name__ == "__main__":
    # Inizializza il preprocessore
    preprocessor = NetCDFPreprocessor(
        root_dir=ROOT_DIR,
        preprocessing_method="filtered"
    )
    
    # ==== OPZIONE 1: Visualizzazione standard ====
    # results = preprocessor.process_file_with_peak_analysis(
    #     "/path/to/single/file.nc",
    #     visualize=True,
    #     visualization_type='standard'  # Visualizzazione base con picco
    # )
    
    # ==== OPZIONE 2: Visualizzazione griglia quadranti dettagliata ====
    # results = preprocessor.process_file_with_peak_analysis(
    #     "/path/to/single/file.nc",
    #     visualize=True,
    #     visualization_type='quadrant_grid'  # Mostra griglia con statistiche
    # )
    
    # ==== OPZIONE 3: Report completo ====
    # results = preprocessor.process_file_with_peak_analysis(
    #     "/path/to/single/file.nc",
    #     visualize=True,
    #     visualization_type='report',  # Report dettagliato
    #     save_reports=True  # Salva i report come immagini
    # )
    
    # ==== OPZIONE 4: Tutte le visualizzazioni ====
    # results = preprocessor.process_file_with_peak_analysis(
    #     "/path/to/single/file.nc",
    #     visualize=True,
    #     visualization_type='all'  # Mostra tutte le visualizzazioni
    # )
    
    # ==== OPZIONE 5: Processa tutti i file e mostra statistiche ====
    all_results = preprocessor.process_all_files(
        max_files=1,  # Limita a 10 file per test
        visualize=True  # Non visualizzare ogni DDM singolarmente
    )

    # ==== OPZIONE 6: Visualizzazione manuale di DDM specifiche ====
    # # Carica un file e estrai alcune DDM
    # with netCDF4.Dataset("/path/to/file.nc", 'r') as f:
    #     output_array, _ = preprocessor.preprocess(f)
    #     
    #     # Estrai alcune DDM non vuote
    #     ddm_list = []
    #     peak_list = []
    #     for i in range(min(6, output_array.shape[0])):
    #         for j in range(min(1, output_array.shape[1])):
    #             ddm = output_array[i, j, :, :]
    #             if not np.all(np.isnan(ddm)):
    #                 peak_coords, _ = preprocessor.find_peak_in_ddm(ddm)
    #                 ddm_list.append(ddm)
    #                 peak_list.append(peak_coords)
    #     
    #     # Visualizza multiple DDM con quadranti
    #     if ddm_list:
    #         fig = preprocessor.visualize_multiple_ddm_quadrants(
    #             ddm_list, peak_list, quadrant_size=(4, 8)
    #         )
    #         plt.show()
    #         
    #         # Crea report per la prima DDM
    #         fig = preprocessor.create_quadrant_analysis_report(
    #             ddm_list[0], peak_list[0], save_path="ddm_analysis_report.png"
    #         )
    #         plt.show()

In [ ]:
    # Processa un singolo file con visualizzazione
results = preprocessor.process_file_with_peak_analysis(
         "/path/to/single/file.nc",
         visualize=True
     )